SAR-CAM Noise Reduction Method - Applying the trained weight on SARMSSD dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



Cloning the SAR-CAM Github Repository

In [ ]:
# Download SAR-CAM repository
!git clone https://github.com/JK-the-Ko/SAR-CAM.git
%cd /content/SAR-CAM


Cloning into 'SAR-CAM'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 87 (delta 48), reused 20 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (87/87), 28.36 KiB | 1.18 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/SAR-CAM


In [ ]:
# install requirements

%pip install -r requirements.txt


In [ ]:
# Modifying the model_parts.py for padding adjustment in the _upsample_ class to handle mismatched tensor sizes during concatenation
%%writefile /content/SAR-CAM/model_parts.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class _conv_(nn.Module) :
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_conv_, self).__init__()

        # Create Layer Instance
        self._conv_ = nn.Conv2d(
                            in_channels = in_channels,
                            out_channels = out_channels,
                            kernel_size = kernel_size,
                            stride = stride,
                            padding = (dilation * (kernel_size - 1)) // 2 ,
                            dilation = dilation,
                            bias = bias
                            )

    def forward(self, x) :
        out = self._conv_(x)

        return out

class _conv_block_(nn.Module) :
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_conv_block_, self).__init__()

        # Create Layer Instance
        self._conv_in_ =  _conv_(in_channels, out_channels, kernel_size, stride, dilation, bias)

    def forward(self, x) :
        out = self._conv_in_(x)
        out = F.leaky_relu(out, 0.2, True)

        return  out

class _context_block_(nn.Module) :
    def __init__(self, in_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_context_block_, self).__init__()

        # Create Layer Instance
        self._conv_in_ = _conv_(in_channels, in_channels, kernel_size, stride, dilation, bias)
        self._d_1_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation, bias)
        self._d_2_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation * 2, bias)
        self._d_3_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation * 3, bias)
        self._d_4_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation * 4, bias)
        self._bottleneck_ = _conv_(in_channels * 4, in_channels, 1, stride, dilation, bias)

    def forward(self, x) :
        out = self._conv_in_(x)
        out = torch.cat([self._d_1_(out), self._d_2_(out), self._d_3_(out), self._d_4_(out)], dim = 1)
        out = self._bottleneck_(out)
        out = out + x

        return out

class _channel_attention_module_(nn.Module) :
    def __init__(self, in_channels, stride, dilation, bias) :
        # Inheritance
        super(_channel_attention_module_, self).__init__()

       # Create Layer Instance
        self._aap_ = nn.AdaptiveAvgPool2d(1)
        self._amp_ = nn.AdaptiveMaxPool2d(1)
        self._conv_ = nn.Sequential(
                            _conv_block_(in_channels, in_channels // 4, 1, stride, dilation, bias),
                            _conv_(in_channels // 4, in_channels, 1, stride, dilation, bias)
                            )

    def forward(self, x) :
        out = self._conv_(self._aap_(x)) + self._conv_(self._amp_(x))
        out = F.sigmoid(out)

        return out

class _spatial_attention_module_(nn.Module) :
    def __init__(self, in_channels, stride, dilation, bias) :
        # Inheritance
        super(_spatial_attention_module_, self).__init__()

        # Create Layer Instance
        self._bottleneck_ = _conv_(2, 1, 7, stride, dilation, bias)

    def forward(self, x) :
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self._bottleneck_(out)
        out = F.sigmoid(out)

        return out

class _ResBlock_CBAM_(nn.Module) :
    def __init__(self, in_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_ResBlock_CBAM_, self).__init__()

        # Create Layer Instance
        self._conv_in_ = _conv_(in_channels, in_channels, kernel_size, stride, dilation, bias)
        self._conv_out_ = _conv_(in_channels, in_channels, kernel_size, stride, dilation, bias)
        self._cam_ = _channel_attention_module_(in_channels, stride, dilation, bias)
        self._sam_ = _spatial_attention_module_(in_channels, stride, dilation, bias)

    def forward(self, x) :
        out = self._conv_in_(x)
        out = out * self._cam_(out)
        out = out * self._sam_(out)
        out = self._conv_out_(out + x)

        return out

class _residual_channel_attention_block_(nn.Module) :
    def __init__(self, in_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_residual_channel_attention_block_, self).__init__()

        # Create Layer Instance
        self._layer_ = _conv_block_(in_channels, in_channels, kernel_size, stride, dilation, bias)
        self._conv_ = nn.Sequential(
                                    nn.AdaptiveAvgPool2d(1),
                                    _conv_block_(in_channels, in_channels // 4, 1, stride, dilation, bias),
                                    _conv_(in_channels // 4, in_channels, 1, stride, dilation, bias),
                                    )

    def forward(self, x) :
        out = self._layer_(x)
        out = out * F.sigmoid(self._conv_(out))
        out = out + x

        return out

class _residual_group_(nn.Module) :
    def __init__(self, in_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_residual_group_, self).__init__()

        # Create Layer Instance
        self._cab_1_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation, bias)
        self._cab_2_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation, bias)
        self._cab_3_ = _residual_channel_attention_block_(in_channels, kernel_size, stride, dilation, bias)
        self._conv_out_ = _conv_(in_channels, in_channels, kernel_size, stride, dilation, bias)

    def forward(self, x) :
        out = self._cab_1_(x)
        out = self._cab_2_(out)
        out = self._cab_3_(out)
        out = self._conv_out_(out)
        out = x + out

        return out

class _upsample_(nn.Module) :
    def __init__(self, scale, in_channels, kernel_size, stride, dilation, bias) :
        # Inheritance
        super(_upsample_, self).__init__()

        # Create Layer Instance
        self._up_ = nn.Sequential(
                            nn.PixelShuffle(scale),
                            _conv_block_(in_channels, in_channels, kernel_size, stride, dilation, bias)
                            )
        self._bottleneck_ = _conv_(in_channels * 2, in_channels, 1, stride, dilation, bias)

    def forward(self, x, skip) :
        out = self._up_(x)

        # Handle mismatched dimensions with padding
        if out.shape[2] != skip.shape[2] or out.shape[3] != skip.shape[3]:
            diff_h = skip.shape[2] - out.shape[2]
            diff_w = skip.shape[3] - out.shape[3]
            out = F.pad(out, (0, diff_w, 0, diff_h))

        out = torch.cat((out, skip), dim=1)
        out = self._bottleneck_(out)

        return out

Overwriting /content/SAR-CAM/model_parts.py


In [ ]:
# Modifying test.py for adapting it to the YOLO11 dataset structure format
%%writefile /content/SAR-CAM/test.py

import argparse
from os import listdir, makedirs
from os.path import join, exists
import shutil

import PIL.Image as pil_image
import PIL.ImageFilter as pil_image_filter

import cv2
import numpy as np
import pandas as pd

import torch
from torchvision import transforms

from model import Model
from utils import calc_psnr, calc_ssim, set_logging, select_device

from tqdm import tqdm

def process_images(noisy_dir, clean_dir, save_dir, model, device, to_tensor, to_pil):
    """Denoise images and save them to the specified directory."""
    makedirs(save_dir, exist_ok=True)
    image_files = [f for f in listdir(noisy_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.tif'))]

    with tqdm(total=len(image_files), desc=f"Processing {noisy_dir}") as pbar:
        for image_file in image_files:
            noisy_path = join(noisy_dir, image_file)
            clean_path = join(clean_dir, image_file)
            save_path = join(save_dir, image_file)

            if not exists(clean_path):
                print(f"Warning: Clean image not found for {image_file}, skipping...")
                continue

            # Load noisy image
            noisy_image = pil_image.open(noisy_path).convert("L")
            tensor_noisy_image = to_tensor(noisy_image).unsqueeze(0).to(device)

            # Apply SAR-CAM model for denoising
            with torch.no_grad():
                pred = model(tensor_noisy_image).detach().cpu()
                pred = torch.clamp(pred, min=0.0, max=1.0)

            # Save denoised image
            denoised_image = to_pil(pred.squeeze(0))
            denoised_image.save(save_path)
            pbar.update()

def copy_labels(label_dir, save_dir):
    """Copy label files to the new directory."""
    if not exists(label_dir):
        return
    makedirs(save_dir, exist_ok=True)
    for label_file in listdir(label_dir):
        src = join(label_dir, label_file)
        dest = join(save_dir, label_file)
        shutil.copy(src, dest)

def main():
    # Argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-name", type=str, default="SAR-CAM")
    parser.add_argument("--weights-dir", type=str, required=True)
    parser.add_argument("--clean-image-dir", type=str, required=True)
    parser.add_argument("--noisy-image-dir", type=str, required=True)
    parser.add_argument("--save-dir", type=str, required=True)
    parser.add_argument("--stack-image", action="store_true")
    parser.add_argument("--device", default="", help="cuda device, i.e. 0 or 0,1,2,3 or cpu")
    args = parser.parse_args()

    # Get Current Namespace
    print(args)

    # Assign Device
    set_logging()
    device = select_device(args.model_name, args.device)

    # Load Model
    model = Model(
        scale=2,
        in_channels=1,
        channels=128,
        kernel_size=3,
        stride=1,
        dilation=1,
        bias=True
    ).to(device)

    model.load_state_dict(torch.load(args.weights_dir, map_location=device))
    model.eval()

    # Create Torchvision Transforms Instance
    to_tensor = transforms.ToTensor()
    to_pil = transforms.ToPILImage()

    # Process train, val, and test datasets
    for subset in ["train", "val", "test"]:
        subset_noisy_dir = join(args.noisy_image_dir, subset, "images")
        subset_clean_dir = join(args.clean_image_dir, subset, "images")
        subset_label_dir = join(args.noisy_image_dir, subset, "labels")

        subset_save_images_dir = join(args.save_dir, subset, "images")
        subset_save_labels_dir = join(args.save_dir, subset, "labels")

        if exists(subset_noisy_dir):
            # Process images
            process_images(subset_noisy_dir, subset_clean_dir, subset_save_images_dir, model, device, to_tensor, to_pil)

        # Copy labels
        copy_labels(subset_label_dir, subset_save_labels_dir)

    # Compute PSNR & SSIM
    image_files = []
    for subset in ["train", "val", "test"]:
        image_dir = join(args.noisy_image_dir, subset, "images")
        if exists(image_dir):
            image_files.extend([join(image_dir, f) for f in listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.tif'))])

    image_name_list, psnr_noisy_list, psnr_denoised_list, ssim_noisy_list, ssim_denoised_list = [], [], [], [], []

    with tqdm(total=len(image_files)) as pbar:
        with torch.no_grad():
            for noisy_image_path in image_files:
                image_name = noisy_image_path.split("/")[-1]
                subset = noisy_image_path.split("/")[-3]  # Get the dataset split (train/val/test)

                clean_image_path = join(args.clean_image_dir, subset, "images", image_name)

                if not exists(clean_image_path):
                    print(f"Warning: Clean image not found for {image_name}, skipping...")
                    continue

                # Load Image
                clean_image = pil_image.open(clean_image_path).convert("L")
                noisy_image = pil_image.open(noisy_image_path).convert("L")

                # Convert Pillow Image to PyTorch Tensor
                tensor_clean_image = to_tensor(clean_image).unsqueeze(0)
                tensor_noisy_image = to_tensor(noisy_image).unsqueeze(0).to(device)

                # Get Prediction
                pred = model(tensor_noisy_image)

                # Assign Device into CPU
                tensor_noisy_image = tensor_noisy_image.detach().cpu()
                pred = pred.detach().cpu()

                # Calculate PSNR & SSIM
                psnr_noisy = calc_psnr(tensor_noisy_image, tensor_clean_image).item()
                psnr_denoised = calc_psnr(pred, tensor_clean_image).item()

                ssim_noisy = calc_ssim(tensor_noisy_image, tensor_clean_image, size_average=True).item()
                ssim_denoised = calc_ssim(pred, tensor_clean_image, size_average=True).item()

                # Append to lists
                image_name_list.append(image_name)
                psnr_noisy_list.append(psnr_noisy)
                psnr_denoised_list.append(psnr_denoised)
                ssim_noisy_list.append(ssim_noisy)
                ssim_denoised_list.append(ssim_denoised)

                # Convert and Save Image
                pred = torch.clamp(pred, min=0.0, max=1.0)
                pred = to_pil(pred.squeeze(0))
                pred.save(join(args.save_dir, subset, "images", image_name))

                # Update TQDM Bar
                pbar.update()

    # Save results as CSV
    df = pd.DataFrame({
        "Image Name": image_name_list,
        "Noisy Image PSNR (dB)": psnr_noisy_list,
        "Noisy Image SSIM": ssim_noisy_list,
        "Denoised Image PSNR (dB)": psnr_denoised_list,
        "Denoised Image SSIM": ssim_denoised_list
    })
    df.to_csv(join(args.save_dir, "image_quality_assessment.csv"), index=False)

    print("Processing complete! Denoised images saved in:", args.save_dir)

if __name__ == "__main__":
    main()


Overwriting /content/SAR-CAM/test.py


In [ ]:
# here below we add our pretrained sar-cam weight that we have obtained from custom revised DOTA 128x128 patch size dataset training
%cd /content/SAR-CAM
!unzip /content/drive/MyDrive/best_model.zip

/content/SAR-CAM
Archive:  /content/drive/MyDrive/best_model.zip
   creating: best_model/
   creating: best_model/sarmssd/
  inflating: best_model/sarmssd/SAR-CAM_best.pth  


In [ ]:
%cd /content/SAR-CAM
!unzip /content/drive/MyDrive/sarmssd.zip

Streaming output truncated to the last 5000 lines.
  inflating: sarmssd/train/labels/P0053_0_800_1800_2600.txt  
  inflating: sarmssd/train/labels/P0053_0_800_2400_3200.txt  
  inflating: sarmssd/train/labels/P0053_0_800_3000_3800.txt  
  inflating: sarmssd/train/labels/P0053_0_800_3600_4400.txt  
  inflating: sarmssd/train/labels/P0053_0_800_4200_5000.txt  
  inflating: sarmssd/train/labels/P0053_0_800_4800_5600.txt  
  inflating: sarmssd/train/labels/P0053_1200_2000_1800_2600.txt  
  inflating: sarmssd/train/labels/P0053_1200_2000_3000_3800.txt  
  inflating: sarmssd/train/labels/P0053_1200_2000_3600_4400.txt  
  inflating: sarmssd/train/labels/P0053_1200_2000_4200_5000.txt  
  inflating: sarmssd/train/labels/P0053_1200_2000_600_1400.txt  
 extracting: sarmssd/train/labels/P0053_1800_2600_1800_2600.txt  
 extracting: sarmssd/train/labels/P0053_1800_2600_2400_3200.txt  
  inflating: sarmssd/train/labels/P0053_2400_3200_0_800.txt  
 extracting: sarmssd/train/labels/P0053_2400_3200_1200

In [ ]:
!python test.py --weights-dir /content/SAR-CAM/best_model/sarmssd/SAR-CAM_best.pth --clean-image-dir /content/SAR-CAM/sarmssd --noisy-image-dir /content/SAR-CAM/sarmssd --save-dir /content/SAR-CAM/denoised_images

Namespace(model_name='SAR-CAM', weights_dir='/content/SAR-CAM/best_model/sarmssd/SAR-CAM_best.pth', clean_image_dir='/content/SAR-CAM/sarmssd', noisy_image_dir='/content/SAR-CAM/sarmssd', save_dir='/content/SAR-CAM/denoised_images', stack_image=False, device='')
SAR-CAM v1.0.0-17-gea5ee3b torch 2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40513.8125MB)

/content/SAR-CAM/test.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the u

In [ ]:
!zip -r /content/sarmssd_sar-cam_denoised.zip /content/SAR-CAM/denoised_images

Streaming output truncated to the last 5000 lines.
  adding: content/SAR-CAM/sarmssd/train/labels/L480.txt (deflated 16%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0064_0_800_1200_2000.txt (deflated 21%)
  adding: content/SAR-CAM/sarmssd/train/labels/000248.txt (deflated 58%)
  adding: content/SAR-CAM/sarmssd/train/labels/L249.txt (deflated 64%)
  adding: content/SAR-CAM/sarmssd/train/labels/001126.txt (deflated 72%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0132_2400_3200_9600_10400.txt (deflated 20%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0135_2400_3200_6600_7400.txt (deflated 18%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0073_600_1400_3600_4400.txt (deflated 17%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0068_1200_2000_4200_5000.txt (deflated 40%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0113_4200_5000_7800_8600.txt (deflated 24%)
  adding: content/SAR-CAM/sarmssd/train/labels/P0116_4200_5000_7200_8000.txt (deflated 78%)
  adding: content/SAR

In [ ]:
%cp /content/sarmssd_sar-cam_denoised.zip /content/drive/MyDrive